Extract the RSEM counts from cancer samples and merge into a dataframe

In [ ]:
import boto3
import pandas as pd

# AWS cilents
s3_client = boto3.client('s3')

# Find filename of all cancer tsv files for input into DESeq
print('Getting df of cancer counts')
cancer_data = []
for obj in s3_client.list_objects(Bucket='zb4171', Prefix='kazoo/data_deseq/')['Contents']:
    obj_key = obj['Key']
    if obj_key.endswith('gene_counts.tsv') and 'failed_runs' not in obj_key:
        sample_id = obj_key.split('data_deseq/')[1].split('/star_rsem/')[0]
        tsv_file = s3_client.get_object(Bucket='zb4171', Key=obj_key)['Body']
        df = pd.read_csv(tsv_file, sep='\t')
        df = df[['gene_id', 'expected_count']].rename(columns={'expected_count': sample_id})
        cancer_data.append(df)


# Make count matrix
print('Make count matrix')
df_counts = pd.DataFrame()
for df in cancer_data:
    if df_counts.empty:
        df_counts = df
    else:
        df_counts = df_counts.merge(df, on='gene_id', how='inner')


# Fill na with 0 and remove rows with all 0
df_counts = df_counts.fillna(0)

df_counts.to_csv('cancer_counts.tsv', sep='\t')